# Integrating Time Points and Time Intervals

<i>Version 1</i>

In [1]:
import qualreas as qr

## References

* Allen, James F. (26 November 1983). <b>"Maintaining knowledge about temporal intervals"</b>. Communications of the ACM. 26 (11): 832–843.
    * [PDF](http://cse.unl.edu/~choueiry/Documents/Allen-CACM1983.pdf)
* Reich, Alfred J. (1994). <b>“Intervals, Points, and Branching Time.”</b> 121-133.
    * Published in TIME 1994: An International Workshop on Temporal Representation and Reasoning.
    * <b>ABSTRACT.</b> This paper extends Allen's interval algebra to include points and either left or right-branching time. The branching time algebras each contain 24 relations: Allen's original 13 relations, 5 more relations resulting from the inclusion of points, and 6 relations because of the inclusion of branching time. The paper also presents: (1) A technique for automatically deriving algebras of relations; (2) a way of representing temporal constraint networks using constraint matrices; and (3) a way of performing constraint propagation using constraint matrix multiplication.
    * <b>PDFs</b> are available at the following locations:
        * [ResearchGate](https://www.researchgate.net/publication/220810644_Intervals_Points_and_Branching_Time) (Includes a supplementary file containing algebras in structured text form.)
        * [SemanticScholar](https://www.semanticscholar.org/paper/Intervals%2C-Points%2C-and-Branching-Time-Reich/a12eca974573c812d696f3d0a58bccb171a8c39d)
        * [Time-94](http://www2.cs.uregina.ca/~temporal/time94/reich.pdf)

### An Interval & Point Algebra

Allen's original algebra of time pertains only to proper intervals.  That is, the domains and ranges of each of his 13 relations are proper time intervals.  Degenerate intervals (i.e., time points) are not permitted.

The Interval & Point algebra, described in [Reich 1994] and illustrated in this notebook, extends Allen's algebra by adding 5 new relations and modifying the domains and ranges of the relations to include, not only proper intervals, but also time points (i.e., degenerate intervals).

<p>Each of the 5 new relations have something to do with time points and so their abbreviated name starts with the letter 'P'.</p>

In [8]:
import os
path = os.path.join(os.getenv('PYPROJ'), 'qualreas/Algebras')
allenX = qr.Algebra(os.path.join(path, 'ExtendedLinearIntervalAlgebra.json'))
allenX.elements

relset(['B', 'BI', 'D', 'DI', 'E', 'F', 'FI', 'M', 'MI', 'O', 'OI', 'PE', 'PF', 'PFI', 'PS', 'PSI', 'S', 'SI'])

Of Allen's original 13 relations, 9 stay the same here with respect to their domains and ranges (that is, they apply to proper intervals only), 4 have their domain, range, or both modified, and 5 are entirely new.  See the last two columns of the table in the next cell, below.

In [11]:
allenX.summary()

  Algebra Name: ExtendedLinearIntervalAlgebra
   Description: Extension of Allen's algebra to include Instants
 Equality Rels: E|PE
     Relations:
            NAME (ABBREV)         CONVERSE (ABBREV)  REFLEXIVE  SYMMETRIC TRANSITIVE   DOMAIN        RANGE
             Before (  B)               After ( BI)    False      False       True    Pt|PInt       Pt|PInt
              After ( BI)              Before (  B)    False      False       True    Pt|PInt       Pt|PInt
             During (  D)            Contains ( DI)    False      False       True    Pt|PInt          PInt
           Contains ( DI)              During (  D)    False      False       True       PInt       Pt|PInt
             Equals (  E)              Equals (  E)     True       True       True       PInt          PInt
           Finishes (  F)         Finished-by ( FI)    False      False       True       PInt          PInt
        Finished-by ( FI)            Finishes (  F)    False      False       True       PInt    

Create the singleton relation sets, {Before} and {Met-By} for this algebra and compute their product. Comparing this to the same calculation for Allen's algebra (previous notebook) we see that an additional relation, PS, appears in the resulting relset.

In [14]:
B = allenX.relset(["B"])
MI = allenX.relset(["MI"])
BxMI = allenX.compose(B, MI)
print(BxMI)

B|D|M|O|PS|S


### Multiplication is Still Not Commutative

In [21]:
MIxB = allenX.compose(MI, B)
print(f"MI x B = {MIxB}")

BxMI = allenX.compose(B, MI)
print(f"B x MI = {BxMI}")

print(f"MI x B == B x MI ? {MIxB == BxMI}")

MI x B = B|DI|FI|M|O|PFI
B x MI = B|D|M|O|PS|S
MI x B == B x MI ? False


In [24]:
F = allenX.relset(["F"])

MIxF = allenX.compose(MI, F)
print(f"MI x F = {MIxF}")

FxMI = allenX.compose(F, MI)
print(f"F x MI = {FxMI}")

print(f"MI x F == F x MI ? {MIxF == FxMI}")

MI x F = MI
F x MI = BI
MI x F == F x MI ? False


### Equality Relations

Allen's algebra has only one identity element, E, because it deals with only one class of temporal entity, the <i>proper interval</i>.

<p>The <i>Interval and Point Algebra</i>, on the other hand, deals with two types of temporal entities, and so it has an equality relation for each one, intervals and points.  Note that intervals here can be degenerate (i.e., equivalent to a point).  This means that the relation <i>E</i> in Allen's algebra is not quite the same as the relation <i>E</i> in the Interval and Point Algebra, since the latter handles degenerate intervals, whereas the former only applies to proper intervals

<p>The equality relations of an algebra are important when propagating contraints, so there is an Algebra property that returns a list of its equality relations:

In [25]:
print(f"The Interval & Point Algebra's equality relations: {allenX.all_equality_relations}")

The Interval & Point Algebra's equality relations: E|PE


## Associativity

### Example: {PS} x {B} x {D}

Associativity does not hold for this example as shown below:

In [41]:
PS = allenX.relset(["PS"])
B = allenX.relset(["B"])
D = allenX.relset(["D"])
PSxB = allenX.compose(PS,B)
print(f"({PS} * {B}) * {D}")
print(f"= {PSxB} * {D}")
print(f"= {allenX.compose(PSxB, D)}")

BxD = allenX.compose(B, D)
print(f"\n{PS} * ({B} * {D})")
print(f"= {PS} * {BxD}")
print(f"= {allenX.compose(PS, BxD)}")

(PS * B) * D
= B * D
= B|D|M|O|PS|S

PS * (B * D)
= PS * B|D|M|O|PS|S
= B|D|PS
